In [1]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from tempfile import TemporaryFile
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [2]:
df=pd.read_csv('train_gue.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2325165 entries, 0 to 2325164
Data columns (total 30 columns):
orderID             object
orderDate           object
articleID           object
colorCode           int64
sizeCode            object
productGroup        float64
quantity            int64
price               float64
rrp                 float64
voucherAmount       float64
customerID          object
paymentMethod       object
returnQuantity      int64
mmdd                object
months              int64
order_total         float64
customer_budget     float64
expense_ratio       float64
unit_price          float64
usual_unit_price    float64
price_diff          float64
after_voucher       float64
order_order         int64
choice_order        int64
articleID_prob      float64
colorCode_prob      float64
customerID_prob     float64
sizeCode_prob       float64
ac_prob             float64
as_prob             float64
dtypes: float64(17), int64(6), object(7)
memory usage: 532.2+ MB


In [3]:
#drop fitur2 yg tipenya object (bkn numerik)

object_columns=df.loc[:, df.dtypes == object].columns
df= df.drop(object_columns, axis=1)

In [4]:
#banyak yg didrop biar ga keberatan memorinya. Yg didrop yg menurutku redundan atau kurang prioritas
#fyi, kalau fitur2 ini didrop dan diprediksi pake LinearRegression (tanpa dibuat PolynomialFeatures nya),
# errornya jadi 83007, naik dr yg sebelumnya 82644
#  tapi entah ini krn fitur2nya tak drop atau krn yg 82644 missing valuenya tak drop (yg 83007 enggak soalnya 
#   fitur ada missing valuenya (productGroup, rrp, sm cvoucherID) udah didrop duluan

df= df.drop(['colorCode','productGroup','months','rrp','price','voucherAmount','after_voucher'], axis=1)

In [6]:
label='returnQuantity'

Xdf=df.drop([label], axis=1)

X=Xdf.values
y=df.returnQuantity.values

In [7]:
#generate polynomial features
#with degree=2, e.g: [x1, x2] --> [1, x1, x2, x1^2, x2^2, x1*x2]

poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [5]:
#kalau ga didelete ngehang, memorinya ga cukup

del df
del X
del y

In [6]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[118437  49227      8      0      0      0]
 [ 31806 148593    123      2      0      0]
 [    26    314    205      0      0      0]
 [     0      6     14      6      0      0]
 [     0      2      0      1      0      0]
 [     0      0      1      3      1      0]]
81585


In [7]:
lr=linear_model.Lasso()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[124486  43183      3      0      0      0]
 [ 52403 128079     41      1      0      0]
 [   117    427      1      0      0      0]
 [     5     21      0      0      0      0]
 [     1      2      0      0      0      0]
 [     1      4      0      0      0      0]]
96384
